In [7]:
###
### Fetch full item catalog
###

import requests
import json
import pandas
import numpy
import datetime

# NOTE: This is not the entire Roblox catalog. It is just the items with limited quantity.
# These are the only items which actually carry IRL value. (Scarcity = value)
# Limiteds can be official Roblox gear or they can be UGC (user generated content).
# Additionally, this set is limited to what rblx.trade tracks. I'm unsure what that includes or excludes.

url = "https://rblx.trade/api/v1/catalog/all"
dir = "./roblox-catalog.json"
try:
    file = open(dir, "r")
    print("catalog is cached")
    catalog = json.loads(file.read())
    file.close()
except:
    catalog_req = requests.get(url)
    print("no cache found at " + url + "; sending request")
    print("status: " + str(catalog_req.status_code))
    catalog = json.loads(catalog_req.text)
    file = open(dir, "w")
    file.write(catalog_req.text)
    file.close()

print("items found: " + str(len(catalog)))

catalog is cached
items found: 2498


In [23]:
###
### Fetch asset prices and volumes
###

import os
import requests
import time
import pandas

df = pandas.DataFrame(catalog)
dir = "./roblox-itemvalues/"
history_api = "https://rblx.trade/api/v1/item/graph/price-and-volume?assetId=_ID_"
details_api = "https://catalog.roblox.com/v1/catalog/items/_ID_/details?itemType=Asset"

def get_history(asset_id, i, total):
    filepath = "{0}/{1}.json".format(dir, str(asset_id))
    
    try:
        file = open(filepath, "r")
        print("({}/{}) #{} (in cache)".format(i, total, asset_id), end="\33[2K\r")
        file.close()
    except:
        print("({}/{}) fetching #{}".format(i, count, asset_id), end="\33[2K\r")
        file = open(filepath, "w")
        fetched = False
        while(not fetched):
            try:
                req = requests.get(history_api.replace("_ID_", str(asset_id)), timeout=2)
                fetched = True
            except:
                time.sleep(2)
        file.write(req.text)
        file.close()
    
def main():
    try:
        os.mkdir(dir)
    except:
        None
    
    count = len(df["roblox_assetid"])
    for (i, asset_id) in enumerate(df["roblox_assetid"]):
        get_history(asset_id, i+1, count)
    

main()